In [2]:
import pandas as pd
import difflib
import re

load census data and state codes

In [3]:
census = pd.read_csv('/home/shumway743/cis5500_group/census/acs_2021.csv')
state_df = pd.read_csv('state_codes.csv')

census_city = census[census.SUMLEVEL.isin({160})]
census_city['area_name'] = census_city.NAME.apply(lambda x: x.split(', ')[0])

census_metro = census[census.SUMLEVEL.isin({310})]
census_metro['state_code'] = census_metro.NAME.apply(lambda x: x.split(', ')[1].split(' ')[0])
census_metro['area_name'] = census_metro.NAME.apply(lambda x: x.split(', ')[0])



/tmp/ipykernel_528/4063471290.py:1: DtypeWarning: Columns (5,39,40) have mixed types. Specify dtype option on import or set low_memory=False.
  census = pd.read_csv('/home/shumway743/cis5500_group/census/acs_2021.csv')
/tmp/ipykernel_528/4063471290.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  census_city['area_name'] = census_city.NAME.apply(lambda x: x.split(', ')[0])
/tmp/ipykernel_528/4063471290.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  census_metro['state_code'] = census_metro.NAME.appl

read and formatt county crime data

In [4]:
county = pd.read_csv('county_crime.csv')
county = county[['county_name','crime_rate_per_100000','FIPS_ST','FIPS_CTY']]
county = county.rename({'county_name':'area_name','crime_rate_per_100000':'crime_rate'},axis=1)
county['crime_index'] = county.crime_rate / county.crime_rate.mean()
county['type'] = 'county'

county.head()

,area_name,crime_rate,FIPS_ST,FIPS_CTY,crime_index,type
0,"St. Louis city, MO",1791.995377,29,510,7.622734,county
1,"Crittenden County, AR",1754.914968,5,35,7.465002,county
2,"Alexander County, IL",1664.700485,17,3,7.081251,county
3,"Kenedy County, TX",1456.310680,48,261,6.194809,county
4,"De Soto Parish, LA",1447.402430,22,31,6.156915,county


read and formatt city crime data

In [5]:
city = pd.read_csv('city_crime.csv')
city['crime_per_100k'] = city.Violent_crime / (city.Population / 100000)
city = city[['State','City','crime_per_100k']]
city['crime_index'] = city.crime_per_100k / city.crime_per_100k.mean()
city['type']='metro'
city = city.rename({'State':'state','City':'area_name'},axis=1)

city['state_lower'] = city.state.str.lower()

city = city.merge(state_df, on='state_lower')
city['name_state_abv'] = city.area_name + ' ' + city.abv
city['name_state'] = city.area_name + ' ' + city.name
city.head()

,state,area_name,crime_per_100k,crime_index,type,state_lower,name,abv,fips,name_state_abv,name_state
0,ALABAMA,Hoover,133.068752,0.518567,metro,alabama,Alabama,AL,1,Hoover AL,Hoover Alabama
1,ALASKA,Anchorage,1244.565236,4.850056,metro,alaska,Alaska,AK,2,Anchorage AK,Anchorage Alaska
2,ALASKA,Bethel,1986.552567,7.741572,metro,alaska,Alaska,AK,2,Bethel AK,Bethel Alaska
3,ALASKA,Bristol Bay Borough,234.741784,0.914786,metro,alaska,Alaska,AK,2,Bristol Bay Borough AK,Bristol Bay Borough Alaska
4,ALASKA,Cordova,0.000000,0.000000,metro,alaska,Alaska,AK,2,Cordova AK,Cordova Alaska


join census data with county crime

In [7]:
dfj = county.merge(census[census.SUMLEVEL == 50][['GEO_ID','STATE','COUNTY']], left_on=['FIPS_ST','FIPS_CTY'], right_on=['STATE','COUNTY'], how='inner')
pd.options.display.max_columns = None
dfj = dfj[['GEO_ID','crime_rate','crime_index']]
county_out = dfj.rename({'crime_rate':'crime_per_100k'},axis=1)
county_out.head()
county_out.to_csv('county_crime_output.csv')

In [17]:
def get_match_metro(x):
    m = difflib.get_close_matches(x['clean_name'].lower(), census_metro[x['abv'] == census_metro['state_code']].clean_name.str.lower(),n=1, cutoff=.9)
    return m[0] if len(m) else None
def get_match_city(x):
    m = difflib.get_close_matches(x['clean_name'].lower(), census_city[x['abv'] == census_city['STUSAB']].clean_name.str.lower(),n=1, cutoff=.9)
    return m[0] if len(m) else None

In [9]:

words = {}
def word_count(x):
    s = re.sub("[^\w' ]", "", x.lower()).split()
    for w in s:
        words[w] = words.get(w,0) + 1
census_city.area_name.apply(word_count)
city.area_name.apply(word_count)
census_metro.area_name.apply(word_count)


drop_words = set([x[0] for x in list(filter(lambda x: x[1] > 1000, words.items()))])
print(drop_words)

def remove_words(x):
    out = x
    for w in list(drop_words):
        out = out.lower().replace(w,'')
    return out

{'cdp', 'city', 'village', 'town', 'borough'}


In [ ]:
census_city['clean_name'] = census_city.area_name.apply(remove_words)
census_metro['clean_name'] = census_metro.area_name.apply(remove_words)
city['clean_name'] = city.area_name.apply(remove_words)

city.clean_name = city.clean_name.str.replace('[^a-zA-Z]', '')
census_metro.clean_name = census_metro.clean_name.str.replace('[^a-zA-Z]', '')
census_city.clean_name = census_city.clean_name.str.replace('[^a-zA-Z]', '')


In [18]:
city['match'] = city.apply(get_match_city, axis=1)
city['match_score'] = city.apply(lambda x: difflib.SequenceMatcher(None, x['clean_name'].lower(), x['match'].lower()).ratio() if x['clean_name'] and x['match'] else None ,axis=1)

city.head()

,state,area_name,crime_per_100k,crime_index,type,state_lower,name,abv,fips,name_state_abv,name_state,clean_name,match,match_score
0,ALABAMA,Hoover,133.068752,0.518567,metro,alabama,Alabama,AL,1,Hoover AL,Hoover Alabama,hoover,hoover,1.0
1,ALASKA,Anchorage,1244.565236,4.850056,metro,alaska,Alaska,AK,2,Anchorage AK,Anchorage Alaska,anchorage,None,NaN
2,ALASKA,Bethel,1986.552567,7.741572,metro,alaska,Alaska,AK,2,Bethel AK,Bethel Alaska,bethel,bethel,1.0
3,ALASKA,Bristol Bay Borough,234.741784,0.914786,metro,alaska,Alaska,AK,2,Bristol Bay Borough AK,Bristol Bay Borough Alaska,bristolbay,None,NaN
4,ALASKA,Cordova,0.000000,0.000000,metro,alaska,Alaska,AK,2,Cordova AK,Cordova Alaska,cordova,cordova,1.0


In [22]:
city['metro_match'] = city.apply(get_match_metro, axis=1)
city['metro_match_score'] = city.apply(lambda x: difflib.SequenceMatcher(None, x['clean_name'].lower(), x['metro_match'].lower()).ratio() if x['clean_name'] and x['metro_match'] else None ,axis=1)

city.head()

,state,area_name,crime_per_100k,crime_index,type,state_lower,name,abv,fips,name_state_abv,name_state,clean_name,match,match_score,metro_match,metro_match_score
0,ALABAMA,Hoover,133.068752,0.518567,metro,alabama,Alabama,AL,1,Hoover AL,Hoover Alabama,hoover,hoover,1.0,None,NaN
1,ALASKA,Anchorage,1244.565236,4.850056,metro,alaska,Alaska,AK,2,Anchorage AK,Anchorage Alaska,anchorage,None,NaN,anchorage,1.0
2,ALASKA,Bethel,1986.552567,7.741572,metro,alaska,Alaska,AK,2,Bethel AK,Bethel Alaska,bethel,bethel,1.0,None,NaN
3,ALASKA,Bristol Bay Borough,234.741784,0.914786,metro,alaska,Alaska,AK,2,Bristol Bay Borough AK,Bristol Bay Borough Alaska,bristolbay,None,NaN,None,NaN
4,ALASKA,Cordova,0.000000,0.000000,metro,alaska,Alaska,AK,2,Cordova AK,Cordova Alaska,cordova,cordova,1.0,None,NaN


In [44]:
city_min = city[['name_state','clean_name','match','match_score','metro_match','metro_match_score','crime_per_100k','crime_index','fips']]
city_out = census_city.merge(city_min, left_on=['STATE','clean_name'],right_on=['fips','clean_name'],how='inner')
#todo: find best match
city_out = city_out[['GEO_ID','crime_per_100k','crime_index','NAME','name_state']]
city_out['match_score'] = city_out.apply(lambda x: difflib.SequenceMatcher(None, x['NAME'].lower(), x['name_state'].lower()).ratio() if x['NAME'] and x['name_state'] else None ,axis=1)
city_out = city_out.sort_values('match_score',ascending=False).drop_duplicates('GEO_ID')
city_out.head()

,GEO_ID,crime_per_100k,crime_index,NAME,name_state,match_score
5182,1600000US4237955,222.995273,0.869010,"Jefferson Hills borough, Pennsylvania",Jefferson Hills Borough Pennsylvania,0.986301
4186,1600000US3618333,34.317090,0.133733,"Cornwall-on-Hudson village, New York",Cornwall-on-Hudson Village New York,0.985915
4249,1600000US3632710,25.265285,0.098459,"Hastings-on-Hudson village, New York",Hastings-on-Hudson Village New York,0.985915
4236,1600000US3630191,0.000000,0.000000,"Great Neck Estates village, New York",Great Neck Estates Village New York,0.985915
4421,1600000US3680181,0.000000,0.000000,"Westhampton Beach village, New York",Westhampton Beach Village New York,0.985507


In [45]:
tmp = city_out.groupby('GEO_ID',as_index=False).count()
tmp = tmp[tmp.NAME > 1]
tmp.count()


GEO_ID            0
crime_per_100k    0
crime_index       0
NAME              0
name_state        0
match_score       0
dtype: int64

In [51]:
city_out.to_csv('place_crime_output.csv')

In [49]:
metro_min = city[['name_state_abv','match','match_score','metro_match','metro_match_score','crime_per_100k','crime_index','fips','abv','clean_name']]
metro_out = census_metro.merge(metro_min, left_on=['state_code','clean_name'],right_on=['abv','clean_name'],how='inner')
#todo: find best match
metro_out = metro_out[['GEO_ID','crime_per_100k','crime_index','NAME','name_state_abv']]

metro_out['match_score'] = metro_out.apply(lambda x: difflib.SequenceMatcher(None, x['NAME'].lower(), x['name_state_abv'].lower()).ratio() if x['NAME'] and x['name_state_abv'] else None ,axis=1)
metro_out = metro_out.sort_values('match_score',ascending=False).drop_duplicates('GEO_ID')
metro_out.head()

,GEO_ID,crime_per_100k,crime_index,NAME,name_state_abv,match_score
494,310M600US47920,275.618375,1.074082,"Washington Court House, OH Micro Area",Washington Court House OH,0.806452
207,310M600US25760,1835.672916,7.153596,"Helena-West Helena, AR Micro Area",Helena-West Helena AR,0.777778
420,310M600US42300,244.843449,0.954152,"Sault Ste. Marie, MI Micro Area",Sault Ste. Marie MI,0.760000
185,310M600US24060,269.272963,1.049354,"Glenwood Springs, CO Micro Area",Glenwood Springs CO,0.760000
104,310M600US17820,585.012342,2.279786,"Colorado Springs, CO Metro Area",Colorado Springs CO,0.760000


In [50]:
tmp = metro_out.groupby('GEO_ID',as_index=False).count()
tmp = tmp[tmp.NAME > 1]
tmp.count()

GEO_ID            0
crime_per_100k    0
crime_index       0
NAME              0
name_state_abv    0
match_score       0
dtype: int64

In [52]:
metro_out.to_csv('metro_crime_output.csv')